# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Importing the dataset
pokemon = pd.read_csv('pokemon.csv')
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [3]:
# your code here
pokemon['Total_check'] = pokemon['HP']+pokemon['Attack']+pokemon['Defense']+pokemon['Sp. Atk']+pokemon['Sp. Def']+pokemon['Speed']
pokemon

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Total_check
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,318
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,405
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,525
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,625
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True,600
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True,700
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True,600
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True,680


In [4]:
def equal(x):
    if x['Total'] == x['Total_check'] :
        return True
    else:
        return False
pokemon['equal_totals'] = pokemon.apply(equal, axis=1)
pokemon

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Total_check,equal_totals
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,318,True
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,405,True
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,525,True
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,625,True
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,309,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True,600,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True,700,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True,600,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True,680,True


In [5]:
#find if any False value is in there
any_false = pokemon.groupby('equal_totals').size()
any_false
#no False values

equal_totals
True    800
dtype: int64

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [10]:
#using getdummies
onehot_types3 = pd.get_dummies(pokemon, prefix=['','Type 1', 'Type 2'])
onehot_types3

,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,...,Type 2_Ghost,Type 2_Grass,Type 2_Ground,Type 2_Ice,Type 2_Normal,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water
0,1,318,45,49,49,65,65,45,1,False,...,0,0,0,0,0,1,0,0,0,0
1,2,405,60,62,63,80,80,60,1,False,...,0,0,0,0,0,1,0,0,0,0
2,3,525,80,82,83,100,100,80,1,False,...,0,0,0,0,0,1,0,0,0,0
3,3,625,80,100,123,122,120,80,1,False,...,0,0,0,0,0,1,0,0,0,0
4,4,309,39,52,43,60,50,65,1,False,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,600,50,100,150,100,150,50,6,True,...,0,0,0,0,0,0,0,0,0,0
796,719,700,50,160,110,160,110,110,6,True,...,0,0,0,0,0,0,0,0,0,0
797,720,600,80,110,60,150,130,70,6,True,...,1,0,0,0,0,0,0,0,0,0
798,720,680,80,160,60,170,130,80,6,True,...,0,0,0,0,0,0,0,0,0,0


In [12]:
onehot_types_final = onehot_types3.drop(['HP','Attack','Defense','Sp. Atk','Sp. Def','Speed','Generation','Legendary','Total_check','equal_totals'], axis = 1)
onehot_types_final

,#,Total,_Abomasnow,_AbomasnowMega Abomasnow,_Abra,_Absol,_AbsolMega Absol,_Accelgor,_AegislashBlade Forme,_AegislashShield Forme,...,Type 2_Ghost,Type 2_Grass,Type 2_Ground,Type 2_Ice,Type 2_Normal,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water
0,1,318,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,2,405,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,3,525,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,3,625,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,4,309,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,600,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
796,719,700,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
797,720,600,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
798,720,680,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [16]:
onehot_correlation = onehot_types_final.corr()
onehot_correlation

,#,Total,_Abomasnow,_AbomasnowMega Abomasnow,_Abra,_Absol,_AbsolMega Absol,_Accelgor,_AegislashBlade Forme,_AegislashShield Forme,...,Type 2_Ghost,Type 2_Grass,Type 2_Ground,Type 2_Ice,Type 2_Normal,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water
#,1.000000,0.119813,0.016513,0.016513,-0.050941,-0.000648,-0.000648,0.043189,0.054063,0.054063,...,0.121081,0.122995,-0.056721,0.050299,0.108329,-0.195128,-0.025565,-0.009633,0.058814,0.023836
Total,0.119813,1.000000,0.017380,0.046889,-0.036916,0.008822,0.038331,0.017675,0.025052,0.025052,...,-0.004885,-0.039224,0.016486,0.100870,-0.013956,-0.067837,0.076054,-0.000512,0.070307,-0.018800
_Abomasnow,0.016513,0.017380,1.000000,-0.001252,-0.001252,-0.001252,-0.001252,-0.001252,-0.001252,-0.001252,...,-0.004721,-0.006354,-0.007567,0.265078,-0.002508,-0.007453,-0.007338,-0.004721,-0.005949,-0.004721
_AbomasnowMega Abomasnow,0.016513,0.046889,-0.001252,1.000000,-0.001252,-0.001252,-0.001252,-0.001252,-0.001252,-0.001252,...,-0.004721,-0.006354,-0.007567,0.265078,-0.002508,-0.007453,-0.007338,-0.004721,-0.005949,-0.004721
_Abra,-0.050941,-0.036916,-0.001252,-0.001252,1.000000,-0.001252,-0.001252,-0.001252,-0.001252,-0.001252,...,-0.004721,-0.006354,-0.007567,-0.004721,-0.002508,-0.007453,-0.007338,-0.004721,-0.005949,-0.004721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Type 2_Poison,-0.195128,-0.067837,-0.007453,-0.007453,-0.007453,-0.007453,-0.007453,-0.007453,-0.007453,-0.007453,...,-0.028118,-0.037839,-0.045064,-0.028118,-0.014935,1.000000,-0.043700,-0.028118,-0.035428,-0.028118
Type 2_Psychic,-0.025565,0.076054,-0.007338,-0.007338,-0.007338,-0.007338,-0.007338,-0.007338,-0.007338,-0.007338,...,-0.027683,-0.037254,-0.044367,-0.027683,-0.014704,-0.043700,1.000000,-0.027683,-0.034880,-0.027683
Type 2_Rock,-0.009633,-0.000512,-0.004721,-0.004721,-0.004721,-0.004721,-0.004721,-0.004721,-0.004721,-0.004721,...,-0.017812,-0.023970,-0.028547,-0.017812,-0.009461,-0.028118,-0.027683,1.000000,-0.022443,-0.017812
Type 2_Steel,0.058814,0.070307,-0.005949,-0.005949,-0.005949,-0.005949,-0.005949,-0.005949,-0.005949,-0.005949,...,-0.022443,-0.030202,-0.035969,-0.022443,-0.011921,-0.035428,-0.034880,-0.022443,1.000000,-0.022443


In [18]:
onehot_correlation2 = onehot_correlation[onehot_correlation['Total'] != 1.0]
onehot_correlation2['Total'].max()

0.19653165607469758

In [20]:
most_likely = onehot_correlation2.loc[onehot_correlation2['Total']== onehot_correlation2['Total'].max()]
most_likely

,#,Total,_Abomasnow,_AbomasnowMega Abomasnow,_Abra,_Absol,_AbsolMega Absol,_Accelgor,_AegislashBlade Forme,_AegislashShield Forme,...,Type 2_Ghost,Type 2_Grass,Type 2_Ground,Type 2_Ice,Type 2_Normal,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water
Type 1_Dragon,0.10937,0.196532,-0.007221,-0.007221,-0.007221,-0.007221,-0.007221,-0.007221,-0.007221,-0.007221,...,-0.027243,-0.036662,0.112272,0.1187,-0.01447,-0.043005,0.085964,-0.027243,-0.034325,-0.027243


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [ ]:
# your code here